In [ ]:
import numpy as np
#sigmoid function 
def sigmoid(x):
    # 对大的正值和负值进行特殊处理，避免溢出
     # 限制输入范围，防止exp计算溢出
    return 1 / (1 + np.exp(-x))

In [67]:
#sigmoid derivative
def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))  # Derivative of sigmoid function: f'(x) = f(x) * (1 - f(x))

In [68]:
import numpy as np
import h5py
import scipy
from PIL import Image
from scipy import ndimage
import data
load_dataset = data.load_dataset

train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()
print("训练集的样本数", train_set_x_orig.shape[0])
print("测试集的样本数", test_set_x_orig.shape[0])
print("train_set_y的维度", train_set_y.shape)
print("test_set_y的维度", test_set_y.shape)
print("train_set_x_orig的维度", train_set_x_orig.shape)
print("test_set_x_orig的维度", test_set_x_orig.shape)

训练集的样本数 209
测试集的样本数 50
train_set_y的维度 (1, 209)
test_set_y的维度 (1, 50)
train_set_x_orig的维度 (209, 64, 64, 3)
test_set_x_orig的维度 (50, 64, 64, 3)


In [69]:
#转换成一维向量
train_x = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
test_x = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T
print(train_x.shape, test_x.shape)

(12288, 209) (12288, 50)


In [70]:
#归一化处理
train_x = train_x/255.
test_x = test_x/255.


让模型训练更稳定：不同像素的数值范围一致，避免因某一通道（或特征）数值过大而主导模型学习，提升梯度下降的收敛效率。
符合模型预期：很多机器学习模型（尤其是神经网络）的激活函数对 [0,1] 区间的输入更友好，能减少数值不稳定的风险。

## 初始化网网络参数

In [71]:
def initialize_with_zeros(shape):
    """
    创建一个形状为(shape,1) 的 w 矩阵，b=0
    """
    w = np.zeros((shape, 1))
    b = 0

    #检查
    assert(w.shape == (shape, 1))
    assert(isinstance(b, float) or isinstance(b, int))

    return w, b

## 前向和方向传播
根据损失函数、前后向传播向量化代码

In [72]:
def propagate(w,b,X,Y):
    Z = np.dot(w.T,X) + b
    A = sigmoid(Z)
    dZ = A - Y
    dw = np.dot(X,dZ.T) / X.shape[1]
    db = np.sum(dZ) / X.shape[1]
    cost = -1*(np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A)) )/ X.shape[1]
    grads = {
        "dw": dw,
        "db": db
    }
    return grads,cost


## 预测函数
利用得出的参数来进行测试 得出准确率

In [73]:
def predict(w, b, X):
    '''
    预测函数
    :param w: 权重数组
    :param b: 偏置
    :param X: 图片的特征数据
    :return:
    Y_predicition: 预测是否为猫，返回值为0或1
    p: 预测为猫的概率
    '''
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)

    A = sigmoid(np.dot(w.T, X) + b)
    for i in range(A.shape[1]):
        if A[0, i] <= 0.5:
            Y_prediction[0, i] = 0
        else:
            Y_prediction[0, i] = 1
    assert (Y_prediction.shape == (1, m))
    return Y_prediction

## 优化过程

In [74]:
def optimize(w, b, X, Y, num_iterations, learning_rate):
    """
    梯度下降算法更新参数
    :param w: 权重数组
    :param b: 偏置bias
    :param X: 图片的特征数据
    :param Y: 图片的标签数据
    :param num_iterations: 优化迭代次数
    :param learning_rate: 学习率
    :return:    params: 优化后的w和b
                costs: 每迭代100次，记录一次成本
    """

    costs = []
    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)
        dw = grads["dw"]
        db = grads["db"]
        w = w - learning_rate * dw
        b = b - learning_rate * db 
        if i % 100 == 0:
            costs.append(cost)
            print("优化%d次后成本是：%f" % (i, cost))

    params = {
        "w": w,
        "b": b
    }
    grads = {
        "dw": dw,
        "db": db
    }
    return params,grads,costs



## 整体逻辑实现

In [75]:
def model(X_train, Y_train, X_test, Y_test, num_iterations=2001, learning_rate=0.5):
    ###开始
    #初始化参数
    w,b=initialize_with_zeros(X_train.shape[0])
    #梯度下降
    params,grads,costs = optimize(w,b,X_train,Y_train,num_iterations,learning_rate)
    
    w = params["w"]
    b = params["b"]
    
    #预测
    Y_prediction_train = predict(w, b, X_train)
    Y_prediction_test  = predict(w, b, X_test)
    ###结束

    print("训练集准确率：{}".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("测试集准确率：{}".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test,
         "Y_prediction_train": Y_prediction_train,
         "w": w,
         "b": b,
         "learning_rate": learning_rate,
         "num_iterations": num_iterations}  
    return d

## 测试

In [79]:
learning_rate = 0.005
num_iterations = 10000
d = model(train_x,train_set_y,test_x,test_set_y,num_iterations,learning_rate)

优化0次后成本是：0.693147
优化100次后成本是：0.584508
优化200次后成本是：0.466949
优化300次后成本是：0.376007
优化400次后成本是：0.331463
优化500次后成本是：0.303273
优化600次后成本是：0.279880
优化700次后成本是：0.260042
优化800次后成本是：0.242941
优化900次后成本是：0.228004
优化1000次后成本是：0.214820
优化1100次后成本是：0.203078
优化1200次后成本是：0.192544
优化1300次后成本是：0.183033
优化1400次后成本是：0.174399
优化1500次后成本是：0.166521
优化1600次后成本是：0.159305
优化1700次后成本是：0.152667
优化1800次后成本是：0.146542
优化1900次后成本是：0.140872
优化2000次后成本是：0.135608
优化2100次后成本是：0.130708
优化2200次后成本是：0.126137
优化2300次后成本是：0.121861
优化2400次后成本是：0.117855
优化2500次后成本是：0.114093
优化2600次后成本是：0.110554
优化2700次后成本是：0.107219
优化2800次后成本是：0.104072
优化2900次后成本是：0.101097
优化3000次后成本是：0.098280
优化3100次后成本是：0.095610
优化3200次后成本是：0.093075
优化3300次后成本是：0.090667
优化3400次后成本是：0.088374
优化3500次后成本是：0.086190
优化3600次后成本是：0.084108
优化3700次后成本是：0.082119
优化3800次后成本是：0.080219
优化3900次后成本是：0.078402
优化4000次后成本是：0.076662
优化4100次后成本是：0.074994
优化4200次后成本是：0.073395
优化4300次后成本是：0.071860
优化4400次后成本是：0.070385
优化4500次后成本是：0.068968
优化4600次后成本是：0.067604
优化4700次后成本是：0.066291
优化48